In [1]:
!pip install pyOpenSSL==22.0.0 --quiet
!pip install kfp-server-api==1.8.2 --quiet
!pip install kfp==1.8.12 --quiet
!pip install kserve==0.10.0 --quiet
!pip install lightgbm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
asgiref 3.8.1 requires typing-extensions>=4; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
azure-core 1.30.1 requires typing-extensions>=4.6.0, but you have typing-extensions 3.10.0.2 which is incompatible.
kfserving 0.5.1 requires azure-storage-blob<=2.1.0,>=1.3.0, but you have azure-storage-blob 12.9.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 1.8.12 requires typing-extensions<4,>=3.7.4; python_version < "3.9", but you have typing-extensions 4.11.0 which is incompatible.
kfserving 0.5.1 requires azure-storage-blob<=2.1.0,>=1.3.0, but you have azure-storage-blob 12.9.0 which is incompatible.


In [2]:
import kfp
from kfp import dsl
import kfp.components as components

In [3]:

from minio import Minio
import numpy as np
import os

minio_client = Minio(
    "minio-service.kubeflow:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)
minio_bucket = "mlpipeline"

objects = minio_client.list_objects(minio_bucket , prefix="skin-cancer/data/")
for obj in objects:
    print(obj.object_name)



skin-cancer/data/HAM10000_metadata.csv
skin-cancer/data/hmnist_28_28_L.csv
skin-cancer/data/hmnist_28_28_RGB.csv
skin-cancer/data/hmnist_8_8_L.csv
skin-cancer/data/hmnist_8_8_RGB.csv
skin-cancer/data/HAM10000_images_part_1/
skin-cancer/data/HAM10000_images_part_2/


In [4]:
def load_data():
    """
    Function to get dataset and load it to minio bucket
    """
    print("getting data")
    from minio import Minio
    import numpy as np
    import os

    minio_client = Minio(
        "minio-service.kubeflow:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )
    minio_bucket = "mlpipeline"
    a = 0 
    object1 = minio_client.list_objects(minio_bucket , prefix="skin-cancer/data/HAM10000_images_part_1/")
    for obj in object1:
        minio_client.fget_object(minio_bucket,obj.object_name,"/tmp/"+obj.object_name)
        a+=1
        if a == 100:
            break
    b = 0
    object2 = minio_client.list_objects(minio_bucket , prefix="skin-cancer/data/HAM10000_images_part_2/")
    for obj in object2:
        minio_client.fget_object(minio_bucket,obj.object_name,"/tmp/"+obj.object_name)
        b+=1
        if b == 100:
            break

    minio_client.fget_object(minio_bucket,"skin-cancer/data/HAM10000_metadata.csv","/tmp/skin-cancer/HAM10000_metadata.csv")

    print(f"Samples in part 1: {len(os.listdir('/tmp/skin-cancer/data/HAM10000_images_part_1/'))}")
    print(f"Samples in part 2: {len(os.listdir('/tmp/skin-cancer/data/HAM10000_images_part_2/'))}")

In [5]:
def build_and_train_model():
    print("processing data")
    import cv2
    import os
    import numpy as np
    import pandas as pd
    from tqdm import tqdm
    from cv2 import imread, resize
    from sklearn.model_selection import train_test_split
    ###
    from minio import Minio
    import numpy as np
    import os
    
    ###
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.models import Sequential, load_model
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    from tensorflow.keras.layers import Dropout, Activation
    from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, \
         Dense, Input, Activation, Dropout, GlobalAveragePooling2D, AveragePooling2D
    from tensorflow.python.keras.utils.np_utils import to_categorical
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.models import Model
    from sklearn.utils.class_weight import compute_class_weight
    from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
    from cv2 import imread, resize # manipulating the images
    from tensorflow.keras.optimizers import Adam

    from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
    ###

    minio_client = Minio(
        "minio-service.kubeflow:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )
    minio_bucket = "mlpipeline"
    a = 0 
    object1 = minio_client.list_objects(minio_bucket , prefix="skin-cancer/data/HAM10000_images_part_1/")
    for obj in object1:
        minio_client.fget_object(minio_bucket,obj.object_name,"/tmp/"+obj.object_name)
        a+=1
        if a == 100:
            break
    b = 0
    object2 = minio_client.list_objects(minio_bucket , prefix="skin-cancer/data/HAM10000_images_part_2/")
    for obj in object2:
        minio_client.fget_object(minio_bucket,obj.object_name,"/tmp/"+obj.object_name)
        b+=1
        if b == 100:
            break
    ###
    minio_client.fget_object(minio_bucket,"skin-cancer/data/HAM10000_metadata.csv","/tmp/skin-cancer/HAM10000_metadata.csv")
    df_skin = pd.read_csv('/tmp/skin-cancer/HAM10000_metadata.csv')
    print(df_skin.info())
    
    # Lesion/dis names are given in the description of the data set.
    lesion_type_dict = {
        'nv': 'Melanocytic nevi',
        'mel': 'Melanoma',
        'bkl': 'Benign keratosis-like lesions ',
        'bcc': 'Basal cell carcinoma',
        'akiec': 'Actinic keratoses',
        'vasc': 'Vascular lesions',
        'df': 'Dermatofibroma'
    }

    lesion_ID_dict = {
        'nv': 0,
        'mel': 1,
        'bkl': 2,
        'bcc': 3,
        'akiec': 4,
        'vasc': 5,
        'df': 6
    }

    # Lesion and it's abbriv.
    lesion_names = ['Melanocytic nevi','Melanoma','Benign keratosis-like lesions ',
                   'Basal cell carcinoma','Actinic keratoses','Vascular lesions',
                   'Dermatofibroma']
    lesion_names_short = ['nv','mel','bkl','bcc','akiec','vasc','df']

    # Maping the lesion type and ID to a dict.
    df_skin['lesion_type']=df_skin['dx'].map(lesion_type_dict)
    df_skin['lesion_ID'] = df_skin['dx'].map(lesion_ID_dict)

    # Display the total found images.
    print('Total number of images', len(df_skin))
    print('The problem is unbalanced, since Melanocytic nevi is much more frequent that other labels')

    # Display the count of each lesion.
    df_skin['lesion_type'].value_counts()
    
    def produce_new_img(img2: cv2) -> tuple:
        """
        function to reproduse a new manipulated (rotating of flipping the original one)
        image from the read one, To increase the dimension of the dataset, avoiding overfitting of a single class.
        """
        imga = cv2.rotate(img2, cv2.ROTATE_90_CLOCKWISE)
        imgb = cv2.rotate(img2, cv2.ROTATE_90_COUNTERCLOCKWISE)
        imgc = cv2.rotate(img2, cv2.ROTATE_180)
        imgd = cv2.flip(img2, 0)
        imge = cv2.flip(img2, 1)
        new_imges = imga, imgb, imgc, imgd ,imge
        return new_imges
    
    x = []          # Hold resized images.
    y = []          # Hold image lesion ID from the data set.

    # Listing all files in the part_1, part_2 dirs
    lista1 = os.listdir('/tmp/skin-cancer/data/HAM10000_images_part_1/')
    lista2 = os.listdir('/tmp/skin-cancer/data/HAM10000_images_part_2/')
    
    output_shape = (128, 128)
    
    # [+] Handling images from part 1 directory
    # for i in tqdm(range(len(lista1))):
    for i in tqdm(range(20)):
        # [+] Features: reading and resize the photo.
        fname_image = lista1[i]
        fname_ID = fname_image.replace('.jpg', '')
        img = imread('/tmp/skin-cancer/data/HAM10000_images_part_1/' + str(fname_image))      # read the image 
        img2 = resize(img, output_shape)                                # resise our image
        x.append(img2)                                                  # append the new image to the list x.

        # Targets: Finding the image lesion ID and append it to the y list.
        output = np.array(df_skin[df_skin['image_id'] == fname_ID].lesion_ID)
        y.append(output[0])

        # add more images for class between 1-6, rotating them 
        if output != 0:
            new_img = produce_new_img(img2)
            for i in range(5):
                x.append(new_img[i])
                y.append(output[0])
    
    # [+] Handling images from part 2 directory
    # for i in tqdm(range(len(lista2))):
    for i in tqdm(range(20)):
        # [+] Features: reading and resize the photo.
        fname_image = lista2[i]
        fname_ID = fname_image.replace('.jpg', '')
        img = imread('/tmp/skin-cancer/data/HAM10000_images_part_2/' + str(fname_image))
        img2 = resize(img, output_shape)
        x.append(img2)

        # Targets: Finding the image lesion ID and append it to the y list.
        output = np.array(df_skin[df_skin['image_id'] == fname_ID].lesion_ID)
        y.append(output[0])

        # [+] Add more images for class between 1-6
        if output != 0:
            new_img = produce_new_img(img2)
            for i in range(5):
                x.append(new_img[i])
                y.append(output[0])
    
    x = np.array(x)
    y = np.array(y)
    print(x.shape, y.shape)
    # convert y (targets) array as required by softmax activation function
    y_train = to_categorical(y, num_classes = 7)

    # split in 80% training and 20% test data
    X_train, X_test, y_train, y_test = train_test_split(x,                  # Images array.
                                                        y_train,            # The training set.
                                                        test_size = 0.20,   # Split data set into 20/80.
                                                        random_state = 50,  # Shuffling number to random the set.
                                                        stratify = y)       # Mix training and test sets.
    # [+] Display the count of train/test data set.
    print('Train dataset shape', X_train.shape, y_train.shape)
    print('Test dataset shape', X_test.shape, y_test.shape)
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
    from tensorflow.keras.layers import Dropout, Activation
    from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, \
         Dense, Input, Activation, Dropout, GlobalAveragePooling2D, AveragePooling2D
    from tensorflow.keras.models import Model
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
    
    vgg = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    x = vgg.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(7, activation='softmax')(x)

    model = Model(vgg.input, x)
    print(model.summary())
    optimizer = Adam(learning_rate= 0.0001)
    model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    datagen = ImageDataGenerator(horizontal_flip=True ,zoom_range=0.15, shear_range=0.15)
    datagen.fit(X_train)
    for layer in model.layers[:-6]:
        layer.trainable = False
    bs = 128
    epochs = 10

    his1 = model.fit(datagen.flow(X_train,y_train),
                        epochs = epochs,
                        batch_size = bs,
                        shuffle = True,
                        validation_data = (X_test, y_test))
    
    from tensorflow import keras
    keras.models.save_model(model,"/tmp/skin-cancer")
    
    ### Save model to minIO
    
    from minio import Minio
    import os

    minio_client = Minio(
            "minio-service.kubeflow:9000",
            access_key="minio",
            secret_key="minio123",
            secure=False
        )
    minio_bucket = "mlpipeline"

    import glob

    def upload_local_directory_to_minio(local_path, bucket_name, minio_path):
        assert os.path.isdir(local_path)

        for local_file in glob.glob(local_path + '/**'):
            local_file = local_file.replace(os.sep, "/") # Replace \ with / on Windows
            if not os.path.isfile(local_file):
                upload_local_directory_to_minio(
                    local_file, bucket_name, minio_path + "/" + os.path.basename(local_file))
            else:
                remote_path = os.path.join(
                    minio_path, local_file[1 + len(local_path):])
                remote_path = remote_path.replace(
                    os.sep, "/")  # Replace \ with / on Windows
                minio_client.fput_object(bucket_name, remote_path, local_file)

    upload_local_directory_to_minio("/tmp/skin-cancer",minio_bucket,"models/skin-cancer/1/") # 1 for version 1
    
    print("Saved model to minIO")
def model_serving():
    """
    Create kserve instance
    """
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1SKLearnSpec
    from datetime import datetime

    namespace = utils.get_default_target_namespace()

    name='skin-cancer'
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version

    isvc = V1beta1InferenceService(api_version=api_version,
                                   kind=constants.KSERVE_KIND,
                                   metadata=client.V1ObjectMeta(
                                       name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                   spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       service_account_name="sa-minio-kserve",
                                       sklearn=(V1beta1SKLearnSpec(
                                           storage_uri="s3://mlpipeline/models/skin-cancer/"))))
    )

    KServe = KServeClient()
    KServe.create(isvc)

In [6]:
import kfp
from kfp import dsl
import kfp.components as components

comp_get_data = components.create_component_from_func(load_data,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
comp_model_building = components.create_component_from_func(build_and_train_model,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0",
                                                           packages_to_install=['opencv-python-headless','tqdm'])
comp_model_serving = components.create_component_from_func(model_serving,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0",
                                                           packages_to_install=['kserve==0.11.2'])

@dsl.pipeline(
    name='skin-cancer',
    description='Detect skin'
)
def output_test(n_loops,methods):
    step1 = comp_get_data()
    
    step2 = comp_model_building()
    step2.after(step1)
    
    step3 = comp_model_serving()
    step3.after(step2)

In [7]:
if __name__ == "__main__":
    client = kfp.Client()

    arguments = {
        "n_loops" : 10,
        "methods": "SVM"
    }

    run_directly = 0
    
    if (run_directly == 1):
        client.create_run_from_pipeline_func(output_test,arguments=arguments,experiment_name="test")
    else:
        kfp.compiler.Compiler().compile(pipeline_func=output_test,package_path='output_test_1.yaml')
        client.upload_pipeline(pipeline_package_path='output_test_1.yaml',pipeline_name="skin_cancer_10")

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


In [2]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1TFServingSpec
from datetime import datetime

In [3]:
namespace = utils.get_default_target_namespace()

In [26]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from datetime import datetime

namespace = utils.get_default_target_namespace()

name='skin-cancer'
kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                                   service_account_name="sa-minio-kserve",
                                   min_replicas = 5,
                                   max_replicas = 7,
                                   sklearn=(V1beta1SKLearnSpec(
                                       storage_uri="s3://mlpipeline/models/skin-cancer/"))))
)

KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-04-30T18:13:25Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:maxReplicas': {},
       'f:minReplicas': {},
       'f:serviceAccountName': {},
       'f:sklearn': {'.': {}, 'f:name': {}, 'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-04-30T18:13:25Z'}],
  'name': 'skin-cancer',
  'namespace': 'kubeflow-user-example-com',
  'resourceVersion': '26482016',
  'uid': '088208c9-824f-4b24-b0a7-61803857d898'},
 'spec': {'predictor': {'maxReplicas': 7,
   'minReplicas': 5,
   'model': {'modelFormat': {'name': 'sklearn'},
    'name': '',
   

In [22]:
from kubernetes.client import V1Container
# Định nghĩa yêu cầu tài nguyên CPU
cpu_request_milli = "5m"  # Yêu cầu 5 millicores

# Tạo đối tượng V1ResourceRequirements với yêu cầu CPU đã định nghĩa
resources = V1ResourceRequirements(requests={'cpu': cpu_request_milli})

# Tạo đối tượng V1beta1Container với cấu hình tài nguyên
container =V1Container(
    resources=resources
)

# Tạo đối tượng V1beta1PredictorSpec với cấu hình container
predictor_spec = V1beta1PredictorSpec(
    service_account_name="sa-minio-kserve",
    sklearn=V1beta1SKLearnSpec(
        storage_uri="s3://mlpipeline/models/skin-cancer/"
    ),
    containers=[container]
)

# Tạo đối tượng V1beta1InferenceServiceSpec với cấu hình predictor
isvc_spec = V1beta1InferenceServiceSpec(
    predictor=predictor_spec
)

# Tạo đối tượng V1beta1InferenceService với cấu hình spec
isvc = V1beta1InferenceService(
    api_version=api_version,
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=name,
        namespace=namespace,
        annotations={'sidecar.istio.io/inject': 'false'}
    ),
    spec=isvc_spec
)

ValueError: Invalid value for `name`, must not be `None`

In [ ]:
###
apiVersion: serving.kserve.io/v1beta1
kind: "InferenceService"
metadata:
  name: skin-cancer
  namespace: kubeflow-user-example-com
  annotations:
    sidecar.istio.io/inject: "false"
spec:
  predictor:
    serviceAccountName: sa-minio-kserve
    tensorflow:
      storageUri: s3://mlpipeline/models/skin-cancer/
      resources:
        limits:
          cpu: '1'
          memory: 2Gi
        requests:
          cpu: 5m
          memory: 2Gi
        
###

In [32]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
import numpy as np
import pandas as pd
import requests
import json
x_number_five = np.array([[52,1,0,1,168,0,1.0,2,2,3]])
KServe = KServeClient()

namespace = utils.get_default_target_namespace()
# isvc_resp = KServe.get("heart-disease", namespace=namespace)
# isvc_url = isvc_resp['status']#['address'] #['url']
# print(isvc_url)

t = x_number_five
print(t)

inference_input = {
  'instances': t.tolist()
}


response = requests.post("http://skin-cancer.kubeflow-user-example-com.svc.cluster.local/v1/models/skin-cancer:predict", json=inference_input)
print(response)
r = json.loads(response.text)
print(r)
# print("Predicted: {}".format(r))

{'components': {'predictor': {'latestCreatedRevision': 'heart-disease-predictor-00001'}}, 'conditions': [{'lastTransitionTime': '2024-04-30T19:47:10Z', 'reason': 'PredictorConfigurationReady not ready', 'severity': 'Info', 'status': 'False', 'type': 'LatestDeploymentReady'}, {'lastTransitionTime': '2024-04-30T19:47:10Z', 'message': 'Revision "heart-disease-predictor-00001" failed with message: 0/1 nodes are available: 1 Insufficient memory. preemption: 0/1 nodes are available: 1 No preemption victims found for incoming pod...', 'reason': 'RevisionFailed', 'severity': 'Info', 'status': 'False', 'type': 'PredictorConfigurationReady'}, {'lastTransitionTime': '2024-04-30T19:47:10Z', 'message': 'Configuration "heart-disease-predictor" does not have any ready Revision.', 'reason': 'RevisionMissing', 'status': 'False', 'type': 'PredictorReady'}, {'lastTransitionTime': '2024-04-30T19:47:10Z', 'message': 'Configuration "heart-disease-predictor" does not have any ready Revision.', 'reason': 'Rev

In [31]:
isvc_resp = KServe.get("heart-disease", namespace=namespace)
print(isvc_resp)

{'apiVersion': 'serving.kserve.io/v1beta1', 'kind': 'InferenceService', 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'}, 'creationTimestamp': '2024-04-30T19:47:07Z', 'finalizers': ['inferenceservice.finalizers'], 'generation': 1, 'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:annotations': {'.': {}, 'f:sidecar.istio.io/inject': {}}}, 'f:spec': {'.': {}, 'f:predictor': {'.': {}, 'f:serviceAccountName': {}, 'f:sklearn': {'.': {}, 'f:name': {}, 'f:storageUri': {}}}}}, 'manager': 'OpenAPI-Generator', 'operation': 'Update', 'time': '2024-04-30T19:47:07Z'}, {'apiVersion': 'serving.kserve.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:finalizers': {'.': {}, 'v:"inferenceservice.finalizers"': {}}}}, 'manager': 'manager', 'operation': 'Update', 'time': '2024-04-30T19:47:07Z'}, {'apiVersion': 'serving.kserve.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:status': {'.': {}, 'f:c